In [1]:
# jsmateo 2025 | Q3-code for Quiz 1

### Part A)


In [2]:
# Imports
import pandas as pd

In [3]:
# Import the data
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving Redacted-Department _ CAD Reconciliation.csv to Redacted-Department _ CAD Reconciliation (3).csv


In [4]:
# Read the file in
data = pd.read_csv(filename, parse_dates=['DISPATCH CREATED DATE', 'ALARM DATE TIME', 'CALL COMPLETE'])

# Range of dispatches
print("Dispatches range:", data['DISPATCH CREATED DATE'].min(), "to", data['DISPATCH CREATED DATE'].max())

# % data missing by column
missing_percent = data.isnull().mean() * 100
print("\n% Missing values by column:")
print(missing_percent)

/tmp/ipython-input-171768104.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv(filename, parse_dates=['DISPATCH CREATED DATE', 'ALARM DATE TIME', 'CALL COMPLETE'])
/tmp/ipython-input-171768104.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv(filename, parse_dates=['DISPATCH CREATED DATE', 'ALARM DATE TIME', 'CALL COMPLETE'])
/tmp/ipython-input-171768104.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv(filename, parse_dates=['DISPATCH CREATED DATE', 'ALARM DATE TIME', 'CALL COMPLETE'])


Dispatches range: 2025-03-24 15:54:00 to 2025-08-31 23:03:00

% Missing values by column:
XREF ID                   0.000000
DISPATCH UNIT             0.000000
DISPATCH CREATED DATE     0.000000
INCIDENT NUMBER           0.000000
1ST UNIT ON SCENE        19.454545
ALARM DATE TIME           1.409091
CALL COMPLETE             1.409091
SHIFT                     3.136364
dtype: float64


Some data issues present are missing information as well as different formats. For missing values the most notable one was "1ST UNIT ON SCENE". One solution would be imputation: using a default value such as "Unknown" for cases where the first unit on scene was not known. Second, I noticed some incident numbers have a different format compared to the rest for whichever reason. I believe the best way handle these would be to just flag and manually change them, as there seems to be very few such cases.

In [7]:
# Assign IDs and clean data
data['ID_NO'] = range(1, len(data)+1)

# Fill missing values with "Unknown"
data['1ST UNIT ON SCENE'] = data['1ST UNIT ON SCENE'].fillna("Unknown")
data['SHIFT'] = data['SHIFT'].fillna("Unknown")

# Omit cases where these dates are missing
data = data.dropna(subset=['ALARM DATE TIME', 'CALL COMPLETE'])

# % data missing by column again
missing_percent = data.isnull().mean() * 100
print("\n% Missing values by column:")
print(missing_percent)


% Missing values by column:
XREF ID                  0.0
DISPATCH UNIT            0.0
DISPATCH CREATED DATE    0.0
INCIDENT NUMBER          0.0
1ST UNIT ON SCENE        0.0
ALARM DATE TIME          0.0
CALL COMPLETE            0.0
SHIFT                    0.0
ID_NO                    0.0
dtype: float64


### Data cleaning strategy
My strategy to clean the data was to simply use default values like "Unknown" for the missing data. As for the missing dates data, as the number of them are so few I believe the best way to handle them would be to simply omit them.

In [8]:
# Average time to resolve a call
data['DURATION'] = data['CALL COMPLETE'] - data['ALARM DATE TIME']
average_duration = data['DURATION'].mean()
print("Average time to resolve a call: ", average_duration)

Average time to resolve a call:  90 days 18:28:08.907330567


In [9]:
# Average number of units sent per call
data['NUM_UNITS'] = data['DISPATCH UNIT'].str.split(',').apply(len)
average_units = data['NUM_UNITS'].mean()
print("Average number of fire units per call:", average_units)

Average number of fire units per call: 1.4402950668510834


In [12]:
# Busiest shift
shift_counts = data['SHIFT'].value_counts()
busiest_shift = shift_counts.idxmax()
print("Totals: ", shift_counts)
print("Busiest shift: ", busiest_shift)

Totals:  SHIFT
A          735
C          719
B          677
Unknown     38
Name: count, dtype: int64
Busiest shift:  A


In [20]:
# Matrix of fire alarms organized by day of week and hour of the day

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
data['DAY_OF_WEEK'] = pd.Categorical(data['DAY_OF_WEEK'], categories=day_order, ordered=True)
data['HOUR'] = data['ALARM DATE TIME'].dt.hour

alarm_matrix = data.pivot_table(
    index='HOUR',
    columns = 'DAY_OF_WEEK',
    values='ID_NO',
    aggfunc='count',
    fill_value=0,
    observed=False
)

alarm_matrix['TOTAL'] = alarm_matrix.sum(axis=1)
alarm_matrix.loc['TOTAL'] = alarm_matrix.sum(axis=0)

print(alarm_matrix)

DAY_OF_WEEK  Monday  Tuesday  Wednesday  Thursday  Friday  Saturday  Sunday  \
HOUR                                                                          
0                 8        7          5         2       3         7       4   
1                 9       10          7         8       4         7      10   
2                 5        4          3         3       3         8      10   
3                 9        9          9         1      10         5       7   
4                 3        4          5         2       7         6       4   
5                 9        3          6         5       5         3       7   
6                 5        5          7         7      11         7       9   
7                14       15          9        10      14         5      11   
8                13       14         18        10       7         6      14   
9                10       17         14        14      16        12      14   
10               20       24         18        15   

In [21]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2169 entries, 0 to 2199
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   XREF ID                2169 non-null   int64          
 1   DISPATCH UNIT          2169 non-null   object         
 2   DISPATCH CREATED DATE  2169 non-null   datetime64[ns] 
 3   INCIDENT NUMBER        2169 non-null   object         
 4   1ST UNIT ON SCENE      2169 non-null   object         
 5   ALARM DATE TIME        2169 non-null   datetime64[ns] 
 6   CALL COMPLETE          2169 non-null   datetime64[ns] 
 7   SHIFT                  2169 non-null   object         
 8   ID_NO                  2169 non-null   int64          
 9   DURATION               2169 non-null   timedelta64[ns]
 10  NUM_UNITS              2169 non-null   int64          
 11  DAY_OF_WEEK            2169 non-null   category       
 12  HOUR                   2169 non-null   int32         

### Part c Unsupervised Learning

In [24]:
# Using K-Means and Agglomerative clustering methods to cluster the above data
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sb

In [26]:
matrix = alarm_matrix.drop('TOTAL', errors='ignore').drop('TOTAL', axis=1, errors='ignore') # Remove totals from the clusters
X = matrix.values

# K-means
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans_labels = kmeans.fit_predict(X)
matrix['KMEANS_CLUSTER'] = kmeans_labels
kmeans_score = silhouette_score(X, kmeans_labels)
print("K-Means Silhouette Score:", kmeans_score)

# Agglomerative clustering (Hierarchical)
hier = AgglomerativeClustering(n_clusters=3)
hier_labels = hier.fit_predict(X)
matrix['HIER_CLUSTER'] = hier_labels
hier_score = silhouette_score(X, hier_labels)
print("Hierarchical Silhouette Score:", hier_score)

K-Means Silhouette Score: 0.3128943414686515
Hierarchical Silhouette Score: 0.3304117548551017


1c) I clustered the fire alarm data based on the matrix we made in part b.Neither method performed significantly better than the other as both had fairly low silhouette scores. This suggests that the clusters have no strong groupings in the dataset based on the attributes.

In [35]:
cluster_summary = matrix.groupby('HIER_CLUSTER').mean()
print(cluster_summary)

DAY_OF_WEEK      Monday    Tuesday  Wednesday   Thursday     Friday  \
HIER_CLUSTER                                                          
0             20.181818  18.090909  18.636364  18.818182  18.090909   
1             11.000000  13.333333  13.000000  10.333333  11.833333   
2              6.857143   6.000000   6.000000   4.000000   6.142857   

DAY_OF_WEEK    Saturday     Sunday  KMEANS_CLUSTER  
HIER_CLUSTER                                        
0             16.636364  14.090909        0.909091  
1             12.833333  11.333333        0.333333  
2              6.142857   7.285714        2.000000  


2c) The clusters seem to represent the peak fire alarm hours  
0 : Peak hours  
1 : Moderate hours  
2 : Off hours